# CAF Slice

Perform one frequency shift $f$ for a cross-correlation,
\begin{equation}
{\displaystyle (f\star g)[f, n]\ = \sum _{m=-\infty }^{\infty }{{f^{\star}[m]}}g[m+n] e^{-j2\pi ft}}
\end{equation}

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from caf_verilog.sim_helper import sim_shift
from gps_helper.prn import PRN
from sk_dsp_comm import sigsys as ss
from sk_dsp_comm import digitalcom as dc

In [ ]:
fs = 625e3
freq_shift = 2e3
center = 300
corr_length = 200
prn = PRN(10)
prn_seq = prn.prn_seq()
prn_seq, b = ss.nrz_bits2(np.array(prn_seq), 5)

In [ ]:
ref, rec = sim_shift(prn_seq, center, corr_length, shift=0, freq_shift=freq_shift, fs=fs, padding=True)
ref = np.array(ref)
rec = np.array(rec)

In [ ]:
plt.plot(ref)
plt.plot(np.array(rec).real)

In [ ]:
rxy, lags = dc.xcorr(ref, rec, corr_length)
plt.plot(lags, abs(rxy))
plt.xlabel("Center Offset (Samples)")
plt.grid();

In [ ]:
t = np.arange(len(ref))
ref_shifted = ref * np.exp(2 * np.pi * (freq_shift / fs) * t * 1j)

In [ ]:
plt.plot(ref_shifted.real)
plt.plot(rec.real)

In [ ]:
rxy, lags = dc.xcorr(ref_shifted, rec, corr_length)
plt.plot(lags, abs(rxy))
plt.xlabel("Center Offset (Samples)")
plt.grid();

In [ ]:
ref_start_shift = [val for val in ref]
for i in range(0, 200):
    ref_start_shift[i + 100] = ref_start_shift[i + 100] * np.exp(2 * np.pi * (freq_shift / fs) * i * 1j)
ref_start_shift = np.array(ref_start_shift)

In [ ]:
plt.plot(ref_start_shift.real)
plt.plot(rec.real)

In [ ]:
rxy, lags = dc.xcorr(ref_start_shift, rec, corr_length)
plt.plot(lags, abs(rxy))
plt.xlabel("Center Offset (Samples)")
plt.grid();

## Dot Product

In [ ]:
from caf_verilog.xcorr import dot_xcorr

In [ ]:
ref, rec = sim_shift(prn_seq, center, corr_length, shift=0, freq_shift=freq_shift, fs=fs)
ref = np.array(ref)
rec = np.array(rec)

In [ ]:
rr = dot_xcorr(ref, rec)
rr = np.array(rr)

In [ ]:
plt.plot(abs(rr))
plt.grid();

In [ ]:
t = np.arange(len(ref))
ref_shifted = ref * np.exp(2 * np.pi * (freq_shift / fs) * t * 1j)

In [ ]:
rr_shifted = dot_xcorr(np.conj(ref_shifted), rec)
rr_shifted = np.array(rr_shifted)

In [ ]:
plt.plot(abs(rr_shifted))
plt.grid();

Instead of the complex conjugate, we can also invert the frequency shift.

In [ ]:
ref_shifted_inv = ref * np.exp(2 * np.pi * (-freq_shift / fs) * t * 1j)

In [ ]:
rr_shifted_inv = dot_xcorr(ref_shifted_inv, rec)
rr_shifted_inv = np.array(rr_shifted_inv)

In [ ]:
plt.plot(abs(rr_shifted_inv))
plt.grid();